In [42]:
import pandas as pd
import numpy as np
from datetime import datetime

In [349]:
kr_coal = pd.read_csv(r".\Git\데이터\국내 사회경제데이터\국내석탄소비량.csv", sep=',',encoding='mbcs')
kr_power = pd.read_csv(r".\Git\데이터\국내 사회경제데이터\국내전력사용량final.csv", index_col = 0)
kr_cpi = pd.read_csv(".\Git\데이터\국내 사회경제데이터\국내소비자물가지수.csv", index_col = 0)
kr_prod = pd.read_csv(".\Git\데이터\국내 사회경제데이터\국내광공업생산지수.csv", index_col = 0, header = [0, 1])
kr_oil = pd.read_csv(".\Git\데이터\국내 사회경제데이터\주유소_평균판매가격_제품별.csv", sep=',',encoding='mbcs')
kr_cosd = pd.read_csv(".\Git\데이터\국내 주가지수\국내코스닥지수.csv", index_col = 0)
kr_cosp = pd.read_csv(".\Git\데이터\국내 주가지수\국내코스피지수.csv", index_col = 0)

### 석탄 데이터 전처리

In [350]:
# 석탄 소비량을 제외한 다른 데이터 제거
kr_coal = kr_coal.iloc[8:, :-2]
kr_coal.head()

,연도,1월소비량(천톤),2월소비량(천톤),3월소비량(천톤),4월소비량(천톤),5월소비량(천톤),6월소비량(천톤),7월소비량(천톤),8월소비량(천톤),9월소비량(천톤),10월소비량(천톤),11월소비량(천톤),12월소비량(천톤)
8,2015,197,139,109,58,25,21,15,27,102,251,275,254
9,2016,170,126,90,39,24,21,14,36,93,195,253,194
10,2017,127,113,78,36,18,14,7,18,105,167,247,149
11,2018,119,98,60,34,20,14,11,16,65,177,173,126
12,2019,96,57,39,28,13,8,7,12,50,107,136,91


In [351]:
# 년/월로 데이터가 구분되도록 melt
kr_coal = kr_coal.melt(id_vars = "연도")

In [352]:
# 월 값만 추출
kr_coal["month"] = kr_coal["variable"].str.replace("월소비량(천톤)", "")

In [353]:
# 기존의 변수 피처 제거
kr_coal = kr_coal.drop("variable", axis = 1)

In [354]:
# kr_coal의 연도, 월 피처를 기반으로 모든 월을 리스트화
month_list = [pd.to_datetime(str(kr_coal.loc[i, "연도"]) + "-" + str(kr_coal.loc[i, "month"])) for i in range(len(kr_coal))]

In [355]:
# month_list를 데이터프레임에 입력
kr_coal["일시"] = month_list

In [356]:
# kr_coal에서 연도, month 피처 제거
kr_coal = kr_coal.drop(["연도", "month"], axis = 1)

In [357]:
# kr_coal에서 date 피처를 인덱스로 설정
kr_coal = kr_coal.set_index("일시")

In [358]:
# 인덱스로 정렬
kr_coal = kr_coal.sort_index()

In [359]:
kr_coal.columns = ["석탄소비량"]

In [360]:
kr_coal

,석탄소비량
일시,
2015-01-01,197
2015-02-01,139
2015-03-01,109
2015-04-01,58
2015-05-01,25
...,...
2023-08-01,5
2023-09-01,18
2023-10-01,77


### 전력 데이터 전처리

In [361]:
# 기존 연도와 월 데이터를 기준으로 년월 생성
kr_power["년월"] = pd.to_datetime(kr_power["년월"])

In [362]:
# 피봇 적용
kr_power = kr_power.pivot(index = "년월", columns = ["그룹", "계약구분"], values = ["사용량(kWh)"])

In [363]:
# 모든 컬럼에 공통이던 0레벨, "사용량(kWh)" 레벨 제거
kr_power.columns = kr_power.columns.droplevel(0)

In [364]:
kr_power.shape

(111, 95)

In [365]:
kr_power.head()

그룹                 강원도                                                  \
계약구분               건설업 공공행정국방사회보장행정          광업      교육서비스업      금융보험업   
년월                                                                       
2015-01-01  22917077.0   57919547.0  32273124.0  41898306.0  6620087.0   
2015-02-01  20319476.0   54811472.0  27792347.0  36905587.0  6357303.0   
2015-03-01  18280250.0   45696558.0  29928413.0  35190402.0  5071082.0   
2015-04-01  16887524.0   42718497.0  30599329.0  32348336.0  4799417.0   
2015-05-01  13734378.0   35339282.0  28773133.0  22137637.0  3935264.0   

그룹                                                                         \
계약구분            농업임업어업       도매소매업 보건업사회복지서비스업     부동산업임대업 사업시설관리사업지원서비스업   
년월                                                                          
2015-01-01  45243558.0  62282031.0  25403089.0  45208282.0      4826861.0   
2015-02-01  44197336.0  59534631.0  24039233.0  42522587.0      4543244.0   
2015-03-01  41313978.0  50798960.0  20249636.0  34870172.0      3497447.0   
2015-04-01  40595623.0  50890380.0  18564924.0  32701230.0      3203699.0   
2015-05-01  37415787.0  48337166.0  15074318.0  28362752.0      2599983.0   

그룹          ...            충청도                                         \
계약구분        ... 사업시설관리사업지원서비스업       숙박음식점업 예술스포츠여가관련서비스업       운수창고업   
년월          ...                                                         
2015-01-01  ...     19093249.0  183345372.0    44986940.0  70905862.0   
2015-02-01  ...     18092346.0  173310534.0    42653471.0  69046662.0   
2015-03-01  ...     15525005.0  146836641.0    37554570.0  66283617.0   
2015-04-01  ...     14506335.0  145868733.0    34382120.0  67618347.0   
2015-05-01  ...     12551162.0  133575242.0    31933530.0  67424411.0   

그룹                                                                             \
계약구분       전기가스증기공기조절공급업  전문과학기술서비스업           제조업       정보통신업 하수폐기처리원료재생환경복원   
년월                                                                              
2015-01-01    88711612.0  72034731.0  4.558029e+09  54591155.0     82808561.0   
2015-02-01    84949131.0  63987246.0  4.064877e+09  53041413.0     76947803.0   
2015-03-01    86025010.0  64707511.0  4.460561e+09  50767982.0     77243360.0   
2015-04-01    88961397.0  61211426.0  4.389469e+09  54109648.0     78708939.0   
2015-05-01   106657955.0  62885310.0  4.338247e+09  54157160.0     78273500.0   

그룹                         
계약구분       협회단체수리기타개인서비스업  
년월                         
2015-01-01     76567615.0  
2015-02-01     72195567.0  
2015-03-01     59391871.0  
2015-04-01     55234915.0  
2015-05-01     45091832.0  

[5 rows x 95 columns]

### 소비자 물가지수 전처리

In [366]:
# 일자 피처를 년-월 형태의 datetime 타입으로 변환
kr_cpi["일자"] = kr_cpi["일자"].map(lambda x: str(x)[:4] + "-" + str(x)[4:])
kr_cpi["일자"] = pd.to_datetime(kr_cpi["일자"])

In [367]:
# 일자 피처명 일시로 변경
kr_cpi.columns = ['일시', '소비자물가', '농축수산물', '공업제품', '집세', '공공서비스', '개인서비스', '근원물가', '생활물가',
       '소비자물가2', '농축수산물2', '공업제품2', '집세2', '공공서비스2', '개인서비스2', '근원물가2',
       '생활물가2']

In [368]:
# 일시 피처 index 설정
kr_cpi = kr_cpi.set_index("일시")

In [369]:
kr_cpi

,소비자물가,농축수산물,공업제품,집세,공공서비스,개인서비스,근원물가,생활물가,소비자물가2,농축수산물2,공업제품2,집세2,공공서비스2,개인서비스2,근원물가2,생활물가2
일시,,,,,,,,,,,,,,,,
2015-01-01,1.0,1.7,0.2,2.7,0.5,1.9,2.6,-0.3,101.0,101.7,100.2,102.7,100.5,101.9,102.6,99.7
2015-02-01,0.6,1.8,-0.8,2.6,0.4,1.9,2.4,-0.6,101.6,103.5,99.4,105.3,100.9,103.8,105.0,99.1
2015-03-01,0.5,-0.1,-0.4,2.5,0.6,1.9,2.2,-0.6,102.1,103.4,99.0,107.8,101.5,105.7,107.2,98.5
2015-04-01,0.4,0.2,-0.5,2.4,0.7,1.9,2.1,-0.7,102.5,103.6,98.5,110.2,102.2,107.6,109.3,97.8
2015-05-01,0.6,2.7,-0.3,2.5,0.7,1.8,2.1,-0.5,103.1,106.3,98.2,112.7,102.9,109.4,111.4,97.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-01,3.2,7.7,2.1,-0.1,1.9,3.9,3.1,3.7,306.6,531.6,261.6,225.8,155.6,405.6,314.1,321.0
2024-01-01,2.8,8.0,1.8,-0.2,2.2,3.5,2.6,3.4,309.4,539.6,263.4,225.6,157.8,409.1,316.7,324.4
2024-02-01,3.1,11.4,2.1,-0.1,2.0,3.4,2.6,3.7,312.5,551.0,265.5,225.5,159.8,412.5,319.3,328.1


### 생산량 전처리

In [370]:
# 인덱스 값을 datetime 타입으로 변경
kr_prod.index = pd.to_datetime(kr_prod.index, format='%Y%m')

In [371]:
kr_prod.shape

(112, 20)

In [372]:
kr_prod.columns = [(loc + "_생산", busi) for loc, busi in kr_prod.columns]

In [373]:
kr_prod.head()

,"(서울_생산, 광업)","(서울_생산, 전기업 및 가스업)","(서울_생산, 제조업)","(서울_생산, 총지수)","(강원도_생산, 광업)","(강원도_생산, 전기업 및 가스업)","(강원도_생산, 제조업)","(강원도_생산, 총지수)","(충청도_생산, 광업)","(충청도_생산, 전기업 및 가스업)","(충청도_생산, 제조업)","(충청도_생산, 총지수)","(전라도_생산, 광업)","(전라도_생산, 전기업 및 가스업)","(전라도_생산, 제조업)","(전라도_생산, 총지수)","(경상도_생산, 광업)","(경상도_생산, 전기업 및 가스업)","(경상도_생산, 제조업)","(경상도_생산, 총지수)"
index,,,,,,,,,,,,,,,,,,,,
2015-01-01,69.784,132.306,91.575,96.561,102.870,110.740,90.726,95.704,101.896,143.507,88.842,89.462,98.204,126.479,108.963,109.892,155.754,124.265,117.375,117.204
2015-02-01,54.985,110.379,82.312,85.762,90.241,94.675,84.073,86.713,73.492,119.975,78.646,79.218,84.658,107.787,94.634,94.899,145.679,106.410,103.844,103.256
2015-03-01,100.000,110.360,98.909,99.327,117.222,100.632,102.319,103.197,126.728,115.714,92.259,91.006,103.324,109.045,114.222,113.283,169.883,112.075,124.380,122.682
2015-04-01,116.752,88.029,96.417,93.739,117.222,86.101,105.242,102.398,98.303,81.356,91.167,90.708,255.811,83.434,112.404,110.654,165.676,99.317,122.411,120.103
2015-05-01,127.544,77.187,91.717,88.347,111.481,82.310,99.798,97.303,176.858,66.096,87.999,86.658,295.693,78.774,111.366,109.173,157.631,89.566,114.359,112.307


### 유가 전처리

In [374]:
kr_oil["구분"] = pd.to_datetime(kr_oil["구분"], format='%Y년%m월%d일')

In [375]:
kr_oil.columns = ['일시', '고급휘발유', '보통휘발유', '자동차용경유', '실내등유']

In [376]:
kr_oil = kr_oil.set_index("일시")

In [377]:
kr_oil

,고급휘발유,보통휘발유,자동차용경유,실내등유
일시,,,,
2015-01-01,1963.95,1587.31,1399.99,1105.58
2015-01-02,1960.73,1583.94,1397.50,1102.38
2015-01-03,1956.82,1580.04,1393.97,1097.43
2015-01-04,1956.43,1576.80,1391.01,1094.75
2015-01-05,1958.08,1574.43,1389.08,1093.74
...,...,...,...,...
2024-05-15,1947.35,1701.02,1543.62,1365.04
2024-05-16,1945.82,1699.61,1541.71,1364.49
2024-05-17,1944.90,1697.68,1539.03,1363.93


### 주가 전처리

In [385]:
kr_cosd = pd.read_csv(".\Git\데이터\국내 주가지수\국내코스닥지수.csv", index_col = 0)
kr_cosp = pd.read_csv(".\Git\데이터\국내 주가지수\국내코스피지수.csv", index_col = 0)

In [386]:
kr_cosd = kr_cosd.iloc[:, 0]
kr_cosp = kr_cosp.iloc[:, 0]

In [391]:
kr_cosd.columns = ["코스닥"]
kr_cosp.columns = ["코스피"]

In [394]:
kr_stock = pd.concat([kr_cosp, kr_cosd], axis = 1)

In [398]:
kr_stock.index = pd.to_datetime(kr_stock.index)

In [407]:
kr_stock

,종가,종가
일자,,
2015-01-02,1926.44,553.73
2015-01-05,1915.75,561.25
2015-01-06,1882.45,558.90
2015-01-07,1883.83,561.32
2015-01-08,1904.65,566.43
...,...,...
2024-05-10,2727.67,864.16
2024-05-13,2727.21,854.43
2024-05-14,2730.30,862.22


In [400]:
date_list = pd.date_range(start = "2015-01-01", end = "2023-12-31")

In [408]:
date_df = pd.DataFrame(date_list)

In [409]:
date_df.columns = ["일자"]

In [410]:
date_df.set_index("일자")

""
일자
2015-01-01
2015-01-02
2015-01-03
2015-01-04
2015-01-05
...
2023-12-27
2023-12-28
2023-12-29


In [412]:
kr_stock = pd.merge(date_df, kr_stock, on = "일자", how = "left")

In [416]:
kr_stock = kr_stock.ffill().bfill()

In [422]:
kr_stock = kr_stock.set_index("일자")

### 데이터 종합 작업

- 3시간 단위로 데이터 생성

In [333]:
# 유가를 제외한 나머지를 병합한 데이터프레임 생성
kr_economy = pd.concat([kr_coal, kr_power, kr_cpi, kr_prod], axis = 1)

In [334]:
# 24년 이후의 데이터 제거
kr_economy = kr_economy.iloc[:-4, :]

In [336]:
# 3시간 단위의 시간 리스트 생성
time_list = pd.date_range(start = "2015-01-01-00:00", end = "2023-12-31-21:00", freq = "3H")

In [337]:
# 3시간 단위의 데이터프레임 생성
time_df = pd.DataFrame(time_list)
time_df.columns = ["일시"]
time_df = time_df.set_index("일시")
time_df.head()

""
일시
2015-01-01 00:00:00
2015-01-01 03:00:00
2015-01-01 06:00:00
2015-01-01 09:00:00
2015-01-01 12:00:00


In [338]:
for column in kr_economy.columns:
    time_df[column] = np.nan

C:\Users\ITSC\AppData\Local\Temp\ipykernel_1496\528975918.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  time_df[column] = np.nan
C:\Users\ITSC\AppData\Local\Temp\ipykernel_1496\528975918.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  time_df[column] = np.nan
C:\Users\ITSC\AppData\Local\Temp\ipykernel_1496\528975918.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(

In [341]:
for date in kr_economy.index:
    time_df.loc[date, :] = kr_economy.loc[date, :]

In [347]:
time_df = time_df.ffill()

In [443]:
time_df.head()

,석탄소비량,"(강원도, 건설업)","(강원도, 공공행정국방사회보장행정)","(강원도, 광업)","(강원도, 교육서비스업)","(강원도, 금융보험업)","(강원도, 농업임업어업)","(강원도, 도매소매업)","(강원도, 보건업사회복지서비스업)","(강원도, 부동산업임대업)",...,"(충청도_생산, 제조업)","(충청도_생산, 총지수)","(전라도_생산, 광업)","(전라도_생산, 전기업 및 가스업)","(전라도_생산, 제조업)","(전라도_생산, 총지수)","(경상도_생산, 광업)","(경상도_생산, 전기업 및 가스업)","(경상도_생산, 제조업)","(경상도_생산, 총지수)"
일시,,,,,,,,,,,,,,,,,,,,,
2015-01-01 00:00:00,197.0,22917077.0,57919547.0,32273124.0,41898306.0,6620087.0,45243558.0,62282031.0,25403089.0,45208282.0,...,88.842,89.462,98.204,126.479,108.963,109.892,155.754,124.265,117.375,117.204
2015-01-01 03:00:00,197.0,22917077.0,57919547.0,32273124.0,41898306.0,6620087.0,45243558.0,62282031.0,25403089.0,45208282.0,...,88.842,89.462,98.204,126.479,108.963,109.892,155.754,124.265,117.375,117.204
2015-01-01 06:00:00,197.0,22917077.0,57919547.0,32273124.0,41898306.0,6620087.0,45243558.0,62282031.0,25403089.0,45208282.0,...,88.842,89.462,98.204,126.479,108.963,109.892,155.754,124.265,117.375,117.204
2015-01-01 09:00:00,197.0,22917077.0,57919547.0,32273124.0,41898306.0,6620087.0,45243558.0,62282031.0,25403089.0,45208282.0,...,88.842,89.462,98.204,126.479,108.963,109.892,155.754,124.265,117.375,117.204
2015-01-01 12:00:00,197.0,22917077.0,57919547.0,32273124.0,41898306.0,6620087.0,45243558.0,62282031.0,25403089.0,45208282.0,...,88.842,89.462,98.204,126.479,108.963,109.892,155.754,124.265,117.375,117.204


In [423]:
kr_daily = pd.concat([kr_stock, kr_oil], axis = 1)

In [426]:
kr_daily = kr_daily.dropna()

In [434]:
kr_final = pd.concat([time_df, kr_daily], axis = 1)

In [436]:
kr_final = kr_final.ffill()

In [439]:
kr_final.to_csv("./한국경제지표종합.csv")
kr_final2 = pd.read_csv("./한국경제지표종합.csv", index_col = 0)
kr_final2.head()

,석탄소비량,"('강원도', '건설업')","('강원도', '공공행정국방사회보장행정')","('강원도', '광업')","('강원도', '교육서비스업')","('강원도', '금융보험업')","('강원도', '농업임업어업')","('강원도', '도매소매업')","('강원도', '보건업사회복지서비스업')","('강원도', '부동산업임대업')",...,"('경상도_생산', '광업')","('경상도_생산', '전기업 및 가스업')","('경상도_생산', '제조업')","('경상도_생산', '총지수')",종가,종가.1,고급휘발유,보통휘발유,자동차용경유,실내등유
2015-01-01 00:00:00,197.0,22917077.0,57919547.0,32273124.0,41898306.0,6620087.0,45243558.0,62282031.0,25403089.0,45208282.0,...,155.754,124.265,117.375,117.204,1926.44,553.73,1963.95,1587.31,1399.99,1105.58
2015-01-01 03:00:00,197.0,22917077.0,57919547.0,32273124.0,41898306.0,6620087.0,45243558.0,62282031.0,25403089.0,45208282.0,...,155.754,124.265,117.375,117.204,1926.44,553.73,1963.95,1587.31,1399.99,1105.58
2015-01-01 06:00:00,197.0,22917077.0,57919547.0,32273124.0,41898306.0,6620087.0,45243558.0,62282031.0,25403089.0,45208282.0,...,155.754,124.265,117.375,117.204,1926.44,553.73,1963.95,1587.31,1399.99,1105.58
2015-01-01 09:00:00,197.0,22917077.0,57919547.0,32273124.0,41898306.0,6620087.0,45243558.0,62282031.0,25403089.0,45208282.0,...,155.754,124.265,117.375,117.204,1926.44,553.73,1963.95,1587.31,1399.99,1105.58
2015-01-01 12:00:00,197.0,22917077.0,57919547.0,32273124.0,41898306.0,6620087.0,45243558.0,62282031.0,25403089.0,45208282.0,...,155.754,124.265,117.375,117.204,1926.44,553.73,1963.95,1587.31,1399.99,1105.58
